In [1]:
import pinecone
import os
import warnings
warnings.filterwarnings("ignore")

C:\Users\youcef\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
pinecone.init(      
	api_key='0f974a83-d911-40bc-916d-8c3356131b3a',      
	environment='gcp-starter'      
)      
index = pinecone.Index('chatbot')

In [3]:
import pandas as pd 
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
info_df = pd.read_json("../data/devfest_chunks.json")

In [5]:
info_df.head()

,0
0,"Event\nDescription\nDevFest 2023, which is sho..."
1,Event\nformat and timeline\nDevfest’23 will be...
2,Event\nParticipants number\nThe Challenges par...
3,Event\nStructure\nThere will be two parts: a h...
4,Event Challenges\nTimeline and place\nDate: Oc...


In [6]:
embeddings = HuggingFaceEmbeddings(model_name="bert-large-uncased")

No sentence-transformers model found with name C:\Users\youcef/.cache\torch\sentence_transformers\bert-large-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\youcef/.cache\torch\sentence_transformers\bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

In [7]:
info_df = info_df.astype(str)
texts = info_df.values.tolist()
docsearch = Pinecone.from_texts([t[0] for t in texts], embeddings, index_name="chatbot")

In [8]:
import requests

In [9]:
import ai21
from dotenv import load_dotenv
load_dotenv()
ai21.api_key = os.getenv("AI21_API_KEY")


In [10]:
docsearch.similarity_search("where devfest will be held",20)

[Document(page_content='Event\nDescription\nDevFest 2023, which is short for "Developer Festival," is a yearly tech event organized by Google Developers Groups.\nAt DevFest, you can meet other tech enthusiasts , learn new things, and stay updated on the latest tech trends. ', metadata={}),
 Document(page_content='Event\nDescription\nDevFest 2023, which is short for "Developer Festival," is a yearly tech event organized by Google Developers Groups.\nAt DevFest, you can meet other tech enthusiasts , learn new things, and stay updated on the latest tech trends. ', metadata={}),
 Document(page_content='Event Hackathon\nWorkshops and mentorship\nWhile there are no workshops scheduled for the hackathon,\r\nYes, there will be mentorship sessions, our mentors will be available to assist you whenever you need guidance or have questions during the hackathon. \r', metadata={}),
 Document(page_content='Event Hackathon\nWorkshops and mentorship\nWhile there are no workshops scheduled for the hackat

In [ ]:
while True:
        question = input("\n ask your question here : ")
        docs = docsearch.similarity_search(question,10)
        answers = []
        for doc in docs:
            answers.append(str(doc.page_content))
        context = " ".join(answers)
        answer = ai21.Answer.execute(context=context,question=question)['answer']
        if(answer==None):
            print("Sorry i can't answer this question, I'm a chatbot dedicated to only answer questions related to devfest")
        else:
            print(answer)